Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: Policy Search

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The player taking the last object wins.

* Task3.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task3.2: An agent using evolved rules
* Task3.3: An agent using minmax
* Task3.4: An agent using reinforcement learning

## Instructions

* Create the directory `lab3` inside the course repo 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.

**Deadline**

T.b.d.


In [12]:
import logging
from collections import namedtuple

In [13]:
Nimply = namedtuple("Nimply", "row, num_objects")

#### **Define Nim class**

In [14]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        """
        Initialize the Nim class by defining:
        - num_rows: the number of rows the game will have
        - k: the maximum number of elements that a player can remove
        """
        self._rows = [i*2 + 1 for i in range(num_rows)]
        self._k = k

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects
        if sum(self._rows) == 0:
            logging.info("Yeuch")

    def display_board(self):
        for i in range(len(self._rows)):
            print(f"row[{i+1}]:\t" + "| " * self._rows[i])

#### Additional utility functions

In [15]:
import functools
from collections import Counter

def calc_nimsum(rows):
    return functools.reduce(lambda a, b: a ^ b, rows)

def reduce_row(board, idx, board_nimsum, odd=None):
    old_value = board._rows[idx]
    if odd == True:
        board._rows[idx] = 0
    elif odd == False:
        board._rows[idx] = 1
    else:
        board._rows[idx] = board._rows[idx] ^ board_nimsum
    return old_value - board._rows[idx]

def all_ones(board):
    counts = Counter(board._rows)
    for idx, cnt in counts.items():
        if idx == 0 or idx == 1:
            continue
        if cnt != 0:
            return False
    return True

def game_over(board, players, turn):
    # The game is over when there is only one object left
    counts = Counter(board._rows)
    logging.debug(f"Count: {counts}")
    for idx, cnt in counts.items():
        if idx != 0 and idx != 1:
            logging.debug("there is at least one heap with a # > 1")
            return False
        if idx == 1 and cnt != 1:
            logging.debug("there are more than one heaps with 1")
            return False
    print(f"Game Over: {players[(turn) % 2]} wins!")
    return True



#### **Expert's Move**

In [16]:
def make_expert_move(board):
    
    board_nimsum = calc_nimsum(board.rows)

    # When the number of heaps with at least 2 objects is equal to 1
    if functools.reduce(lambda acc, el: acc + 1 if el > 1 else acc, board._rows, 0) == 1:
        # Count the number of heaps with one object
        # if the number is odd...
        if functools.reduce(lambda acc, el: acc + 1 if el == 1 else acc, board._rows, 0) % 2 != 0:
            # ...take all but one of the "non equal to one" heap
            for i in range(len(board._rows)):
                if board._rows[i] != 0 and board._rows[i] != 1:
                    ply = Nimply(i, board.rows[i]-1)
                    #removed = reduce_row(board, i, board_nimsum, odd=False)
                    print(f"Removed {board.rows[i]-1}, from row {i+1}")
                    return ply
        # if the number is even...
        else:
             # ...take all of the "non equal to one" heap
            for i in range(len(board._rows)):
                if board._rows[i] != 0 and  board._rows[i] != 1:
                    ply = Nimply(i, board.rows[i])
                    #removed = reduce_row(board, i, board_nimsum, odd=True)
                    print(f"Removed {board.rows[i]}, from row {i+1}")
                    return ply

    # Check if all the remaining heaps are composed of one element
    if all_ones(board):
        for i in range(len(board._rows)):
            if board._rows[i] == 1:
                #board.nimming(Nimply(i, 1))
                ply = Nimply(i, 1)
                print(f"Removed 1, from row {i+1}")
                return
            

    chosen_idx = None
    for i in range(len(board._rows)): 
        logging.debug(f"nimsum ({board_nimsum}) XOR row[{i}] ({board._rows[i]}) = {board_nimsum ^ board._rows[i]}")
        if board_nimsum ^ board._rows[i] < board._rows[i]:
            chosen_idx = i
            break

    if chosen_idx is None:
        logging.debug(f"No ideal move found")
        # TODO: imporove strategy (take largest and remove one)
        idx = board._rows.index(max(board._rows))
        ply = Nimply(idx, 1)
        #board.nimming(Nimply(idx, 1))
        print(f"Removed 1, from row {idx+1}")
        pass
    else:
        #removed = reduce_row(board, chosen_idx, board_nimsum)
        num_to_remove = board.rows[chosen_idx] - (board.rows[chosen_idx] ^ board_nimsum)
        ply = Nimply(chosen_idx, num_to_remove)
        print(f"Removed {num_to_remove}, from row {chosen_idx+1}")
        return ply

#### **Human Move** (requires input)

In [17]:
def make_human_move(board):
    selected_row = -1
    while selected_row < 0 or selected_row > len(board._rows) or board._rows[selected_row] == 0:
        selected_row = int(input(f"Select heap from which you want to remove objects [1-{len(board._rows)}]")) - 1
        if selected_row < 0 or selected_row > len(board._rows) or board._rows[selected_row] == 0:
            print(f"Invalid row number (the row number might be correct, but there are no objects left in row)\n")
    
    num_objs = 0
    while num_objs <= 0 or num_objs > board._rows[selected_row]:
        num_objs = int(input(f"How many objects do you want to remove [1-{board._rows[selected_row]}]?"))
        if num_objs <= 0 or num_objs > board._rows[selected_row]:
            print(f"Invalid number of objects to be removed\n")
    
    board.nimming(Nimply(selected_row, num_objs))
    print(f"You have removed {num_objs}, from row {selected_row+1}")
    return

### **Play Expert PC against Human Game**

In [18]:
import time

def play_nim(num_of_heaps, first='human'):
    board = Nim(num_of_heaps)
    board.display_board()

    players = ['pc', 'human']

    if first == 'human':
        turn = 1
    else:
        turn = 0

    while not game_over(board, players, turn):
        if players[turn] == 'pc':
            time.sleep(1)
            board_nimsum = calc_nimsum(board._rows)
            make_expert_move(board, board_nimsum)
            board.display_board()
        if players[turn] == 'human':
            make_human_move(board)
            board.display_board()
        print("\n", end="\r")
        turn = (turn + 1) % 2

    return



In [19]:
logging.getLogger().setLevel(logging.DEBUG)
play_nim(4, first='pc')

DEBUG:root:Count: Counter({1: 1, 3: 1, 5: 1, 7: 1})
DEBUG:root:there is at least one heap with a # > 1


row[1]:	| 
row[2]:	| | | 
row[3]:	| | | | | 
row[4]:	| | | | | | | 


DEBUG:root:nimsum (0) XOR row[0] (1) = 1
DEBUG:root:nimsum (0) XOR row[1] (3) = 3
DEBUG:root:nimsum (0) XOR row[2] (5) = 5
DEBUG:root:nimsum (0) XOR row[3] (7) = 7
DEBUG:root:No ideal move found
DEBUG:root:Count: Counter({1: 1, 3: 1, 5: 1, 6: 1})
DEBUG:root:there is at least one heap with a # > 1


Removed 1, from row 4
row[1]:	| 
row[2]:	| | | 
row[3]:	| | | | | 
row[4]:	| | | | | | 



DEBUG:root:Count: Counter({1: 1, 0: 1, 5: 1, 6: 1})
DEBUG:root:there is at least one heap with a # > 1


...

You have removed 4, from row 3
row[1]:	| 
row[2]:	
row[3]:	| 
row[4]:	| | | | 



DEBUG:root:Count: Counter({1: 2, 0: 2})
DEBUG:root:there are more than one heaps with 1


Removed 4, from row 4
row[1]:	| 
row[2]:	
row[3]:	| 
row[4]:	



DEBUG:root:Count: Counter({0: 3, 1: 1})


You have removed 1, from row 1
row[1]:	
row[2]:	
row[3]:	| 
row[4]:	

Game Over: pc wins!
